In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))


In [ ]:

from stable_diffusion.pipeline import generate
from PIL import Image
from transformers import CLIPTokenizer
import torch
from stable_diffusion.utils.model_loader import preload_models_from_standard_weights
import numpy as np
import imageio


In [32]:

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# DEVICE = "cpu"  # Forcing CPU usage
print(f"Using device: {DEVICE}")


Using device: cuda


In [33]:
tokenizer = CLIPTokenizer("../dataa/vocab.json", merges_file="../dataa/merges.txt")
model_file = "../dataa/v1-5-pruned-emaonly.ckpt"
models = preload_models_from_standard_weights(model_file, DEVICE)

In [72]:

# text to image
prompt = "make it cute cartoon character"
do_cfg = True
cfg_scale=7.5

#image to image

image_path = "../image/babygirl.png"
input_image = Image.open(image_path).convert("RGB")
# image = np.array(input_image)
# print(image.shape)

strength = 0.5 # between 0 and 1

sampler = "ddpm"
num_inference_steps = 50
seed = 42

output_image = generate(
    prompt = prompt,
    uncon_prompt = "should not be blurry",
    strength = strength,
    do_cfg = do_cfg,
    cfg_scale = cfg_scale,
    sampler_name= sampler,
    n_inference_steps = num_inference_steps,
    seed = seed,
    device = DEVICE,
    idle_device="cpu",
    model = models,
    tokenizer = tokenizer,
    video = False,
    input_image = input_image,


)


100%|██████████| 25/25 [00:01<00:00, 12.76it/s]


In [76]:
output_image.shape
output_image.dtype

dtype('uint8')

In [ ]:
def save_video(images, out_path="out.mp4", fps=24):
    """
    images: list or np.ndarray of frames, each frame HxWxC (uint8 RGB)
    """
    with imageio.get_writer(out_path, fps=fps, codec="libx264", quality=8) as w:
        for im in output_image:
            a = np.asarray(im)
            if a.dtype != np.uint8:
                a = np.clip(a, 0, 255).round().astype(np.uint8)
            if a.ndim == 2:
                a = np.stack([a]*3, axis=-1)
            w.append_data(a)


In [52]:

# usage:
save_video(output_image, "out.mp4", fps=24)